In [1574]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [1575]:
num_simu = 10 # number of total simulations, smaller number for testing, larger number for data generation.
#np.random.seed(69420)

To begin, we set up and make some assumptions of our model. In this particular case:
- We have one singular flock that has both chickens and ducks in it.
- Within this flock, we categorise the chickens into unvaccinated chickens, sentinel chickens (used to detect outbreaks), and vaccinated chickens. We assume that the sentinel chickens are unvaccinated (thus fully susceptible to HPAI), and the vaccinated chickens cannot be infected at all.
- We fix the value of total chicken population to be 3000, with 30 sentinels. We may vary the total duck, vaccinated chicken population, and testing period for our exploratory analysis.

In [1576]:
######## Changable Values ########
# set number of species and flocks
num_flocks = 1 # One single flock, need to reflect in beta, sigma, and gamma 
num_species = 4 # chicken, sentinel chicken, vaccinated chicken, duck

tot_chicken_popul = 3000 # total chicken population
tot_duck_popul = 3000 # <---- set total duck population (vary between [300, 1500, 3000, 5000])
vaccinated = 0 # <---- choose how many chickens to be vaccinated (vary between [0, 1500, 2250, 2700])

######## Surveillance Strategy ########
surveillance = 30 # how many chickens are sentinel birds / how many chickens to randomly sample
testing_period = 7 # how many days to test the sentinel birds / do random testing. 

We will consider an SEEIIR model where we will split each infected bird into a symptomatic or asymptomatic case. The ability to infect, the latency period, and infectious period can different. Chickens are more likely to be symptomatic while ducks are more likely to be asymptomatic.

In [1577]:
######## Changable Values ########
# Here we may adjust the key parameters for the simulation
same_species_symptomatic_infection_rate = 1.6
same_species_asymptomatic_infection_rate = 0.3
different_species_symptomatic_infection_rate = 1.6
different_species_asymptomatic_infection_rate = 0.3

chicken_symptomatic_latency_period = 1.1
duck_symptomatic_latency_period = 1.1
chicken_asymptomatic_latency_period = 1
duck_asymptomatic_latency_period = 1

chicken_symptomatic_infectious_period = 1.4
duck_symptomatic_infectious_period = 1.4
chicken_asymptomatic_infectious_period = 15
duck_asymptomatic_infectious_period = 15

chicken_symptomatic_prob = 0.95
duck_symptomatic_prob = 0.05

# the habitable area of one flock, for density dependence
farm_area = 5000

We set up the initial population accordingly. We assume that one symptomatic chicken is in the flock at the beginning.

In [1578]:
######## set initial conditions ######## 

# set initial conditions 

# the following convention will be used: first dimension will represent which flock, second is which species, third is the compartment.
init_val = np.zeros((num_flocks, num_species, 6)) # Six possible compartment: S, E_S, E_A, I_S, I_A, R

# first let all birds to start susceptible, also choose population size here.
init_val[:,0,0] += tot_chicken_popul # <---- set total chicken population
init_val[:,3,0] += tot_duck_popul # <---- set total duck population

init_val[0,0,0] -= surveillance
init_val[0,1,0] += surveillance # chicken under surveillance 

init_val[0,0,0] -= vaccinated
init_val[0,2,0] += vaccinated # vaccinated chicken


# store the total population for each flock and each species
tot_popul = init_val[:,:,0].copy()


# then choose a bird to be exposed (symptomatic), here we assume it to be a chicken
init_val[0,0,0] -= 1
init_val[0,0,1] += 1


# this is the maximum number of events that would occur, typically the number will not be reached, but 
# for diseases that does not die out this is necessary to not fall into an infinite while loop.
max_events = 500000

In [1579]:
# initialise the infection rate tensor
beta_S = np.zeros((num_flocks, num_species, num_flocks, num_species))

beta_S[:, :2, :, :2] = same_species_symptomatic_infection_rate # within-chicken infection
beta_S[:, :2, :, 3] = different_species_symptomatic_infection_rate # chicken-to-duck infection
beta_S[:, 3, :, :2] = different_species_asymptomatic_infection_rate # duck-to-chicken infection
beta_S[:, 3, :, 3] = same_species_symptomatic_infection_rate # within-duck infection

beta_A = np.zeros((num_flocks, num_species, num_flocks, num_species))

beta_A[:, :2, :, :2] = same_species_asymptomatic_infection_rate # within-chicken infection
beta_A[:, :2, :, 3] = different_species_asymptomatic_infection_rate # chicken-to-duck infection
beta_A[:, 3, :, :2] = different_species_asymptomatic_infection_rate # duck-to-chicken infection
beta_A[:, 3, :, 3] = same_species_asymptomatic_infection_rate # within-duck infection

# add within-flock density dependence
for n in range(num_flocks):
    beta_S[n, :, n, :] *= np.sum(tot_popul, axis=1)[n] / farm_area
    beta_A[n, :, n, :] *= np.sum(tot_popul, axis=1)[n] / farm_area
# add between-flock frequency dependence
for n in range(num_flocks):
    for m in range(num_flocks):
        beta_S[n, :, m, :] /= np.sum(tot_popul, axis=1)[n]
        beta_A[n, :, m, :] /= np.sum(tot_popul, axis=1)[n]
        if n != m:
            beta_S[n, :, m, :] /= 10
            beta_A[n, :, m, :] /= 10

# latency and infectious period
latency_period_S = np.array([chicken_symptomatic_latency_period, chicken_symptomatic_latency_period, 1, duck_symptomatic_latency_period])
sigma_S = 1 / latency_period_S
latency_period_A = np.array([chicken_asymptomatic_latency_period, chicken_asymptomatic_latency_period, 1, duck_asymptomatic_latency_period])
sigma_A = 1 / latency_period_A

infectious_period_S = np.array([chicken_symptomatic_infectious_period, chicken_symptomatic_infectious_period, 1, duck_symptomatic_infectious_period])
gamma_S = 1 / infectious_period_S
infectious_period_A = np.array([chicken_asymptomatic_infectious_period, chicken_asymptomatic_infectious_period, 1, duck_asymptomatic_infectious_period])
gamma_A = 1 / infectious_period_A

# probability of displaying symptoms
p_S = np.array([chicken_symptomatic_prob, chicken_symptomatic_prob, 0, duck_symptomatic_prob])
p_A = np.ones(num_species) - p_S


In [ ]:
######## Define update rules to be used in the Gillespie Algorithm ########

# def S_to_E(current_val, symptomatic = True, tot_popul=tot_popul, beta_S=beta_S, beta_A=beta_A, p_S=p_S, p_A=p_A, num_flocks=num_flocks, num_species=num_species):
#     output_matrix = np.sum(np.sum(beta_S * current_val[:,:,3].reshape(num_flocks, num_species, 1, 1) + beta_A * current_val[:,:,4].reshape(num_flocks, num_species, 1, 1), axis=1), axis=1) #/ np.sum(tot_popul, axis=1).reshape(num_flocks, 1)
#     if symptomatic:
#         return output_matrix * current_val[:,:,0] * p_S
#     else:
#         return output_matrix * current_val[:,:,0] * p_A

def S_to_E(a, b, current_val, symptomatic = True, tot_popul=tot_popul, beta_S=beta_S, beta_A=beta_A, p_S=p_S, p_A=p_A, num_flocks=num_flocks, num_species=num_species):
    val = 0
    for i in range(num_flocks):
        for j in range(num_species):
            val += (beta_S[i,j,a,b] * current_val[i,j,3] + beta_A[i,j,a,b] * current_val[i,j,4]) 

    if symptomatic:
        val = val * current_val[a,b,0] * p_S[b]
    else:
        val = val * current_val[a,b,0] * p_A[b]
    return val

def E_to_I(a, b, current_val, symptomatic = True, tot_popul=tot_popul, sigma_S=sigma_S, sigma_A=sigma_A, num_flocks=num_flocks, num_species=num_species):
    if symptomatic:
        return current_val[a,b,1] * sigma_S[b]
    else:
        return current_val[a,b,2] * sigma_A[b]

def I_to_R(a, b, current_val, symptomatic = True, tot_popul=tot_popul, gamma_S=gamma_S, gamma_A=gamma_A, num_flocks=num_flocks, num_species=num_species):
    if symptomatic:
        return current_val[a,b,3] * gamma_S[b]
    else:
        return current_val[a,b,4] * gamma_A[b]

# def E_to_I(current_val, symptomatic = True, tot_popul=tot_popul, sigma_S=sigma_S, sigma_A=sigma_A, num_flocks=num_flocks, num_species=num_species):
#     if symptomatic:
#         return current_val[:,:,1] * sigma_S
#     else:
#         return current_val[:,:,2] * sigma_A


# def I_to_R(current_val, symptomatic = True, tot_popul=tot_popul, gamma_S=gamma_S, gamma_A=gamma_A, num_flocks=num_flocks, num_species=num_species):
#     if symptomatic:
#         return current_val[:,:,3] * gamma_S
#     else:
#         return current_val[:,:,4] * gamma_A

In [1581]:

######## Gillespie Algorithm ########

def Gillespie_simu(max_events=max_events, init_val=init_val, tot_popul=tot_popul, 
                   beta_S=beta_S, beta_A=beta_A, sigma_S=sigma_S, sigma_A=sigma_A,
                   gamma_S=gamma_S, gamma_A=gamma_A, p_S=p_S, p_A=p_A, num_flocks=num_flocks, 
                   num_species=num_species):

    # initialise the event count and current values

    num_event = 0
    current_val = init_val.copy()

    # set the time and state sequence
    t = [0] + [None] * max_events
    y = [init_val] + [None] * max_events


    while (num_event < max_events) and (np.sum(current_val[:,:,1:5]) > 0): # stop the loop if: 1. maximum event number is reached, or 2. no more infections can possibly occur.
        
        num_event += 1

        ##### create an event tensor ####

        # all_events = np.zeros((num_flocks, num_species, 6)) # six types of update rules in total
        # all_events[:,:,0] = S_to_E(current_val, True)
        # all_events[:,:,1] = S_to_E(current_val, False)
        # all_events[:,:,2] = E_to_I(current_val, True)
        # all_events[:,:,3] = E_to_I(current_val, False)
        # all_events[:,:,4] = I_to_R(current_val, True)   
        # all_events[:,:,5] = I_to_R(current_val, False)

        all_events = np.zeros((num_flocks, num_species, 6)) # six types of update rules in total
        for i in range(num_flocks):
            for j in range(num_species):
                all_events[i, j, 0] = S_to_E(i, j, current_val, True)
                all_events[i, j, 1] = S_to_E(i, j, current_val, False)
                all_events[i, j, 2] = E_to_I(i, j, current_val, True)
                all_events[i, j, 3] = E_to_I(i, j, current_val, False)
                all_events[i, j, 4] = I_to_R(i, j, current_val, True)
                all_events[i, j, 5] = I_to_R(i, j, current_val, False)

        # store total rate to rescale later
        tot_rate = np.sum(all_events)
        
        # do a time leap
        
        r1 = np.random.uniform()
        t[num_event] = t[num_event-1] - np.log(r1) / tot_rate
        
        # then choose events, first choose the type of events (S to E_S, S to E_A, E_S to I_S, E_A to I_A, I_S to R, or I_A to R)
        
        r2 = np.random.uniform()

        for event in range(6):
            if r2 < np.sum(all_events[:,:,0:event+1]) / tot_rate:
                type_event = event
                break

        # then choose which flock gets updated
        
        r3 = np.random.uniform()
        spec_event_rate = np.sum(all_events[:,:,type_event]) # total rate of a specific event occurring

        for i in range(num_flocks):
            if r3 < np.sum(all_events[0:i+1,:,type_event]) / spec_event_rate:
                flock_to_update = i
                break

        # finally choose which species get updated

        r4 = np.random.uniform()
        spec_event_flock_rate = np.sum(all_events[flock_to_update,:,type_event])

        for j in range(num_species):
            if r4 < np.sum(all_events[flock_to_update,0:j+1,type_event]) / spec_event_flock_rate:
                species_to_update = j
                break

        # do the updating
        if type_event == 0:
            current_val[flock_to_update, species_to_update, 0] -= 1
            current_val[flock_to_update, species_to_update, 1] += 1
        if type_event == 1:
            current_val[flock_to_update, species_to_update, 0] -= 1
            current_val[flock_to_update, species_to_update, 2] += 1
        if type_event == 2:
            current_val[flock_to_update, species_to_update, 1] -= 1
            current_val[flock_to_update, species_to_update, 3] += 1
        if type_event == 3:
            current_val[flock_to_update, species_to_update, 2] -= 1
            current_val[flock_to_update, species_to_update, 4] += 1
        if type_event == 4:
            current_val[flock_to_update, species_to_update, 3] -= 1
            current_val[flock_to_update, species_to_update, 5] += 1
        if type_event == 5:
            current_val[flock_to_update, species_to_update, 4] -= 1
            current_val[flock_to_update, species_to_update, 5] += 1

        # store the updated value

        y[num_event] = current_val.copy()

    # get rid of none value if there is any:
    t = np.array(t[0:num_event+1])
    y = np.array(y[0:num_event+1])

    return t, y # y format: [time, flock, species, compartment]


In [1582]:
simu_params = [{
'num_simu': num_simu,
'num_flocks': num_flocks,
'num_species': num_species,
'tot_chicken_popul': tot_chicken_popul,
'tot_duck_popul': tot_duck_popul,
'vaccinated': vaccinated,
'surveillance': surveillance,
'testing_period': testing_period,
'same_species_symptomatic_infection_rate': same_species_symptomatic_infection_rate,
'same_species_asymptomatic_infection_rate': same_species_asymptomatic_infection_rate,
'different_species_symptomatic_infection_rate': different_species_symptomatic_infection_rate,
'different_species_asymptomatic_infection_rate': different_species_asymptomatic_infection_rate,
'chicken_symptomatic_latency_period': chicken_symptomatic_latency_period,
'duck_symptomatic_latency_period': duck_symptomatic_latency_period,
'chicken_asymptomatic_latency_period': chicken_asymptomatic_latency_period,
'duck_asymptomatic_latency_period': duck_asymptomatic_latency_period,
'chicken_symptomatic_infectious_period': chicken_symptomatic_infectious_period,
'duck_symptomatic_infectious_period': duck_symptomatic_infectious_period,
'chicken_asymptomatic_infectious_period': chicken_asymptomatic_infectious_period,
'duck_asymptomatic_infectious_period': duck_asymptomatic_infectious_period,
'chicken_symptomatic_prob': chicken_symptomatic_prob,
'duck_symptomatic_prob': duck_symptomatic_prob,
'farm_area': farm_area} for _ in range(num_simu)]

# File to save the simulations
output_file = 'simulation_results.h5'

with h5py.File(output_file, 'w') as f:
    for i, params in enumerate(simu_params):
        t, y = Gillespie_simu()
        # Create a group for each simulation
        sim_group = f.create_group(f"simulation_{i+1}")

        # Store time and state data
        sim_group.create_dataset("time", data=t)
        sim_group.create_dataset("state", data=y)

        param_group = sim_group.create_group("parameters")
        for key, value in params.items():
            if isinstance(value, np.ndarray):
                param_group.create_dataset(key, data=value)
            else:
                param_group.attrs[key] = value  # Store scalars/strings as attributes



In [1583]:
# with h5py.File(output_file, 'r') as f:
#     for sim_name in f:
#         print(f"Results for {sim_name}:")
        
#         sim_group = f[sim_name]
#         t = sim_group['time'][:]
#         y = sim_group['state'][:]
#         params = {key: sim_group['parameters'][key][:] if key in sim_group['parameters'] 
#                   else sim_group['parameters'].attrs[key]
#                   for key in sim_group['parameters']}